In [38]:
import os
from os.path import join, getsize
pathdict = {}
for d in os.listdir("C:/Writing Programs/Optical-Modeling/Materials/"):
    pathdict[d]=[]
    for file in os.listdir("C:/Writing Programs/Optical-Modeling/Materials/"+d):
        split = os.path.splitext(file)
        if split[1] == ".csv":
            pathdict[d].append(split[0])
pathdict

{'Dielectric': ['BK7', 'Sapphire', 'SLG'],
 'Metal': [],
 'Other': [],
 'Semiconductor': ['Air', 'CdSe', 'CdTe', 'TCO']}

In [21]:
os.listdir("C:/Writing Programs/Optical-Modeling/Materials/Dielectric")

['BK7.csv', 'BK7.jmp', 'Sapphire.csv', 'SLG.csv', 'Soda-Lime Glass.jmp']

In [33]:
x = []
x.append("fact")
x

['fact']